# Example: The retinal model system

In this tutorial, we demonstrate how to load and analyze surface hopping trajectory data for a retinal model system. 

Our goal is to illustrate a full workflow: importing trajectories, performing essential post-processing, and extracting three key reaction coordinates, which have been identified to comprehensively describe the photoinduced $S_0\rightarrow~S_1$ relaxation and the *cis/trans*-isomerization around the carbon-carbon double bond:

- a **torsion angle** around the central carbon-carbon double bond ($\varphi_{\text{C-C=C-C}}$),
- the **bond-length alternation (BLA)** of the full chromophore, and
- the **out-of-plane movement of the hydrogens (HOOP)** that are connected to the isomerizing C=C bond.

By the end of this guide, you will be able to compute these features from trajectory data and use them for mechanistic interpretation or downstream machine-learning applications.



In [ ]:
import pandas as pd
import seaborn as sns

import shnitsel as st

#specific code for plotting within this tutorial
import plots_retinal as pr

## 1) Import and preparation of data
### 1.1) Loading data

In [ ]:
dt = st.io.read('tut_data/traj_I02.nc', kind='shnitsel')
dt

## 1.2) Preparing the data

For pre-processing, we herein only slice out the dataset from the datatree and assign a mol object to the dataset.

In [ ]:
from shnitsel.data.shnitsel_db.db_function_decorator import concat_subtree

ds = concat_subtree(dt['I02'])
ds

## 2) Computing Descriptors from Internal Coordinates

In the next step, we **flag selected internal coordinates of a molecule**. Flagging identifies which bonds, bond angles, and dihedral angles are considered active for subsequent analysis.

The flagging procedure returns dictionaries of internal coordinates (keys: `'bonds'`, `'angles'`, and `'dihedrals'`), where only those belonging to a specified substructure are marked as active. Substructures can be defined either by SMARTS patterns or by explicit tuples of atom indices.
If no substructure is specified, the entire molecule is flagged, yielding all intrinsic coordinates.

In a subsequent step, the resulting dictionaries are passed to the geocalc module, which computes the corresponding geometric properties from XYZ structures along the trajectories.

## 2.1) Selecting Internal Coordinates

There ate two main strategies to select internal coordinates of interest for further processing:

If only specific internal coordinates are required, individual bonds (minimum 2 atoms), bond angles (minimum 3 atoms), or torsion angles (minimum 4 atoms) can be flagged directly using the functions provided in `shnitsel.geo.geomatch`.

<details>
<summary><strong>Example: flagging C=C double bonds</strong></summary>
<br>
The following example flags all C=C double bonds in a molecule:
<br>

```python
from shnitsel.geo.geomatch import flag_bonds
d_flag, img = flag_bonds(mol, smarts='C=C', draw=True)
```
</details>
<br>

When bonds, bond angles, and torsions of a specific molecular region are of interest, the `shnitsel.geo.geomatch.flag_bats` function can be used. Substructures can be specified either by atom indices or by SMARTS patterns.

<details>
<summary><strong>Example: Flagging bonds, angles and torsions</strong></summary>
<br>
Example for flagging the torsion angle around the central C=C double bond using only carbon atoms, using a tuple of atom indices:
<br>

```python
from shnitsel.geo.geomatch import flag_bats
idxs_phi1 = (3,5,7,9)
d_flag, img_flag = flag_bats(mol, t_idxs=idxs_phi1, draw=True)
```
<br>
Same as above, using SMARTS:

```python
from shnitsel.geo.geomatch import flag_bats
smarts_phi1 = '[#6;D3]C=C[#6;D3]'
d_flag, img_flag = flag_bats(mol, smarts=smarts_phi1, draw=True)
```
</details>
<br>

In the following, we employ the `flag_bats_multiple` function, to account for the three geometric features of interest. These include:

- a **torsion angle** around the central carbon-carbon double bond ($\varphi_{\text{C-C=C-C}}$),
- the **bond-length alternation (BLA)** of the full chromophore, and
- the **out-of-plane movement of the hydrogens (HOOP)** that are connected to the isomerizing C=C bond

These features are defined by flagging the central CC=CC and corresponding HC=CH torsion angles, as well as the chromophore with alternating single and double bonds, using three distinct SMARTS patterns.

In [ ]:
from shnitsel.filtering.structure_selection import StructureSelection

base_selection = StructureSelection.init_from_dataset(ds)

smarts_phi1 = '[#6;D3]C=C[#6;D3]'
smarts_phi2 = '[#1][#6;D3;H1]=[#6;D3;H1][#1]'
smarts_bla = '[#6,#7]=[#6][#6]=[#6][#6]=[#6,#7]'

phi1_selection = base_selection.select_bats(smarts_phi1)
phi2_selection = base_selection.select_bats(smarts_phi2)
bla_selection = base_selection.select_bats(smarts_bla)

phi1_selection.draw(flag_level=2)

In [ ]:

phi2_selection.draw(flag_level=2)

In [ ]:
bla_selection.draw(flag_level=2)

From the PCA plot, we can support that there are actually trajectories that isomerize:

In [ ]:
from shnitsel.vis.plot import biplot_kde

kde_data = biplot_kde(ds, 3,5,7,9)

## 2.2) Computing the Descriptors

After flagging the relevant internal coordinates, the corresponding geometric descriptors (bond lengths, bond angles, and torsion angles) are computed directly from the molecular geometries.

The values are obtained using the functions provided in `shnitsel.geo.geocalc`, with the previously generated flag dictionaries defining which internal coordinates are evaluated. This separation of coordinate selection and value computation enables a flexible and modular workflow.

In [ ]:
from shnitsel.geo.geocalc import get_dihedrals, get_max_chromophor_BLA

# compute tagged torsion angles
arr_phi1 = get_dihedrals(ds.atXYZ, phi1_selection, signed=False).values.reshape(-1)
arr_phi2 = get_dihedrals(ds.atXYZ, phi2_selection, signed=False).values.reshape(-1)

# compute HOOP by subtracting phi2 from phi1 for every data point
arr_hoop = (arr_phi1 - arr_phi2)

# compute BLA for tagged chromophor
arr_bla = get_max_chromophor_BLA(ds.atXYZ, bla_selection).values.reshape(-1)

Now, we combine information on the times, active states, identifiers, energies in $S_0$ and S$_1$ and the respective energy gap with the three descriptors in a pandas.DataFrame for plotting purposes.

In [ ]:
df_all = pd.DataFrame(
    {'time': ds.time.values,
     'astate': ds.astate.values,
     'trajid': ds.trajid.values,
     'E_S0': ds.energy.values[0,:],
     'E_S1': ds.energy.values[1,:],
     'gap': ds.energy.values[1,:]-ds.energy.values[0,:],
     "dihedral": arr_phi1,
     "HOOP": arr_hoop,
     "BLA": arr_bla})

df_all.head(n=3)

Using HoloViews (https://holoviews.org/), we can explore, e.g., how the HOOP coordinate evolves over time for each trajectory, with the trajectory colored continuously by the dihedral angle, to enable an intuitive extrapolative visualization.

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

key_dimensions   = [('time', 'time / fs'), ('trajid', 'Trajectory ID')]
value_dimensions = [('dihedral', 'Dihedral / °'), ('HOOP', 'HOOP / °'), ('BLA', 'BLA / Bohr')]
macro = hv.Table(df_all, key_dimensions, value_dimensions)

# Build a curve that *includes* dihedral as a value dimension
curve = macro.to.curve(
    kdims='time / fs',
    vdims=['HOOP / °', 'Dihedral / °']
)

unem_scatter = macro.to.scatter('time / fs', ['HOOP / °', 'Dihedral / °'])

# Line plot with color mapped to dihedral
(curve * unem_scatter).opts(
    opts.Curve(color='k'),
    opts.Scatter(
        color='Dihedral / °',cmap='vanimo', size=10, frame_width=600, fontsize={'labels': 14, 'ticks': 14}
    )
)

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

key_dimensions   = [('time', 'time / fs'), ('trajid', 'Trajectory ID')]
value_dimensions = [('dihedral', 'Dihedral / °'), ('gap', 'E$_01$ / eV')]
macro = hv.Table(df_all, key_dimensions, value_dimensions)

# Build a curve that *includes* dihedral as a value dimension
curve = macro.to.curve(
    kdims='time / fs',
    vdims=['Dihedral / °','E$_01$ / eV']
)

unem_scatter = macro.to.scatter('time / fs', ['Dihedral / °', 'E$_01$ / eV'])

# Line plot with color mapped to dihedral
(curve * unem_scatter * hv.HLine(90)).opts(
    opts.Curve(color='k'),
    opts.HLine(color='k', line_width=2,line_dash='dashed'),
    opts.Scatter(
        color='E$_01$ / eV',cmap='cividis', size=10, frame_width=600, fontsize={'labels': 14, 'ticks': 14}
    )
)

From the color-coding it is apparent, in which trajectories, isomerization occurs and that we can classify the trajectories based on the dihedral angle in the final frames, i.e. see that trajectories 1, 2, 12 and 14 reach a region with dihedral angles below 90° in a region of greater energy gap between $S_0$ and $S_1$ (yellow vs. region of hopping in blue).

Another variant for plotting is given below, where we plot the change of the S$_1$-$S_0$ energy gap, the BLA and HOOP descriptor over time (color coded by the central dihedral).
By comparing e.g. trajectory #1 and #9 it is apparent, that isomerization occurs only in trajectory #1.

In [ ]:
pr.plot_stack(df_all, trajid_list=[1,9])

To pin this finding into a structural reason, we align the trajectories to the time of the last $S_1\rightarrow{}S_0$ hop.
From this visualization it is clearly apparent, that the difference in reaction outcome can be associated with the HOOP motion, which goes towards negative values in case of no isomerization but positive values for the isomerizing case (trajectory #1).

In [ ]:
df_all_aligned, hop_times = pr.align_trajectories_to_last_hop(df_all, from_state=2, to_state=1)
pr.plot_stack(df_all_aligned, time='time_aligned', trajid_list=[1,9])

Now we can inspect the pairwise correlations between the key properties at the last $S_1 \rightarrow S_0$ hopping point to evaluate whether our conclusions hold across the dataset.

To do this, we first assign labels to each trajectory based on its final dihedral angle, indicating whether it underwent isomerization. We then extract the data corresponding to the hopping points and plot the pairwise relationships using a pairplot. From this visualization, it becomes clear that in most trajectories where no isomerization occurs, the HOOP angle remains near zero, while the central torsion angle stays between 100° and 180°.

In [ ]:
df_hops = pr.label_trajectory(df_all_aligned, lower_bound=70, upper_bound=110)
df_hops_plot = pr.get_last_hops_from_states(df_hops)
df_hops_plot.head(n=15)

In [ ]:
# pairplot
st_blue = (44/255, 62/255, 80/255)
st_yellow = (196/255, 160/255, 0/255)

g = sns.pairplot(
    df_hops_plot,
    vars=["dihedral", "BLA", "HOOP", "gap"],
    hue="isomerization",
    palette={"E-to-Z": st_blue, "E-to-E": st_yellow, 'undetermined': 'lightgray'},
    diag_kind='hist', diag_kws={'multiple':'stack', 'alpha':0.7, 'bins':10},
    plot_kws={"s": 60, "edgecolor": None, "alpha": 1},
    corner=True,
)

g._legend.set_bbox_to_anchor((0.5, 0.8))
g._legend.set_loc('upper center')